# Lab 3: Scoring Agent (Notebook)

This notebook builds a Strands Agent that converts detected impairments into a numeric risk score and rationale. It mirrors the structure from Lab 2, reusing environment setup and tooling patterns.

You will:
- Install dependencies and load environment variables saved in Lab 1 (`../.env`)
- Define tools (`kb_search` for Life; `calculator` for debits/credits)
- Build a Strands Scoring Agent using the provided prompts (Life and P&C)
- Run a local evaluation on a sample impairments payload and print JSON output
- (Optional) Package and deploy to Bedrock AgentCore Runtime and invoke


In [ ]:
%pip install -qU strands-agents bedrock-agentcore bedrock-agentcore-starter-toolkit python-dotenv boto3


In [ ]:
# 1) Load environment and set up AWS clients
import os, json, logging
from pathlib import Path
from dotenv import load_dotenv
import boto3

logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load ../.env created in Lab 1
env_path = Path('../.env')
if env_path.exists():
    load_dotenv(env_path)
    logger.info(f"Loaded env from {env_path}")
else:
    logger.warning("../.env not found. Ensure Lab 1 was completed or set env vars manually.")

REGION = os.getenv('REGION') or boto3.session.Session().region_name
BEDROCK_KB_ID = os.getenv('BEDROCK_KB_ID')
BEDROCK_KB_ARN = os.getenv('BEDROCK_KB_ARN')
FOUNDATION_MODEL_ID =  os.getenv('BEDROCK_SCORING_MODEL_ID', "us.anthropic.claude-3-7-sonnet-20250219-v1:0")

session = boto3.session.Session(region_name=REGION)
kb_runtime = session.client('bedrock-agent-runtime')

print("✅ Environment loaded")
print(f"   Region: {REGION}")
print(f"   KB ID: {BEDROCK_KB_ID}")
print(f"   Model: {FOUNDATION_MODEL_ID}")


# Creating the Scoring Agent 
In the next few cells, we will set up the scoring agent. We will use the Strands Agents framework to build it. 

### Strands Agents: what they are and how we use them here
Strands is a lightweight Python framework for building tool-using AI agents you can run locally and deploy to Bedrock AgentCore. An agent is defined by:
- System prompt: sets the agent’s role (senior underwriter) and operating rules.
- Tools (declared with `@tool`): explicit capabilities the agent is allowed to call (e.g., `kb_search` to ground in the underwriting manual; `calculator` for deterministic math).
- Model: the LLM that reasons, plans, and invokes tools to complete the task.

Why this pattern fits underwriting scoring:
- Policy grounding: the agent retrieves guidance from the Knowledge Base rather than relying on recall, keeping decisions aligned with the manual.
- Governed capabilities: it can only use the tools you expose, creating a clear audit surface.
- Determinism where it matters: debits/credits are summed via a calculator tool to avoid “creative math.”
- Reproducibility and deployment: the same agent you test locally packages cleanly to Bedrock AgentCore for scale, logging, and integration with your workflow/UI.

Outcome:
- The agent ingests impairments and scoring factors, looks up the relevant policy sections, calculates per-impairment subtotals with citations, and returns a clean JSON payload with a final total score and human-readable reasoning that downstream systems can consume.


In [ ]:
# 2) Define tools mirroring the scoring lambda (Life-only)
from strands import Agent, tool
from strands.models import BedrockModel
from typing import List

@tool
def kb_search(canonical_term: str):
    """Return markdown for the top KB hit from Bedrock knowledge base."""
    kb_id = BEDROCK_KB_ID
    if not kb_id:
        return "Knowledge base not configured."
    try:
        resp = kb_runtime.retrieve(
            knowledgeBaseId=kb_id,
            retrievalQuery={'text': canonical_term},
            retrievalConfiguration={'vectorSearchConfiguration': {'numberOfResults': 1}}
        )
        results = resp.get('retrievalResults') or []
        if not results:
            return "No matching documents found."
        content = results[0].get('content') or {}
        return content.get('text') or content.get('text_markdown') or ""
    except Exception as e:
        return f"KB retrieval error: {e}"

@tool
def calculator(values: List[float]):
    """Sum a list of numbers for debits (positive) and credits (negative)."""
    try:
        if not isinstance(values, list):
            return 0
        total = 0.0
        for v in values[:100]:
            try:
                total += float(v)
            except Exception:
                continue
        return total
    except Exception:
        return 0

MODEL_ID = FOUNDATION_MODEL_ID
model = BedrockModel(model_id=MODEL_ID)


### Scoring prompt: role, tools, and structure
This prompt defines the underwriter role and a disciplined scoring workflow: retrieve policy from the Knowledge Base, analyze scoring factors against rating tables, compute a subtotal with the calculator tool, and explain the rationale. The structure keeps scores grounded in the underwriting manual and auditable, and the final JSON contract makes the output machine‑consumable for the workflow and UI.


In [ ]:
# 3) Prompt (Life-only; copied exactly from scoring lambda example)
LIFE_PROMPT = """You are a senior life insurance underwriter specializing in risk assessment scoring. Your job is to calculate a risk score for an application based on a list of identified impairments and their scoring factors.

You will be given a JSON array of impairments. For each impairment in the input list, you must perform the following steps in sequence:

1. **Lookup**: Call the `kb_search` tool using the impairment's `impairment_id` as the `canonical_term`. This returns the authoritative underwriting manual section.

2. **Analyze**: Carefully read the returned markdown. Use the `scoring_factors` provided for the impairment to find the correct debits and credits in the rating tables. For example, a `blood_pressure` of "128/92 mmHg" and `age` of 41 falls into the "141-150/91-95" row for the "Age 40-60" column in the hypertension manual, which indicates a debit between +25 and +50. Use the lower value if a range is given.

3. **Calculate Subtotal**: Create a list of all numerical debits (positive numbers) and credits (negative numbers) you identified. Pass this list to the `calculator` tool to get a `sub_total` for the impairment.

4. **Explain**: After calculating the subtotal, you must generate a detailed `reason` string explaining exactly how you arrived at that score, citing the specific scoring factors, table values, and modifying factors used.

Repeat this entire process for every impairment in the input list.

Once you have a `sub_total` for all impairments, create a final list containing all the individual sub-totals. Call the `calculator` tool one last time with this list to get the final `total_score`.

Finally, structure your entire response as a single JSON object. Do not include any other text or explanation outside of the final JSON block.

Your output must be in this exact format:
```json
{
  "total_score": 100,
  "impairment_scores": [
    {
      "impairment_id": "hypertension",
      "sub_total": 50,
      "reason": "Based on the underwriting manual entires for Hypertension, Debit of +25 for BP 128/92 at age 41. Debit of +25 for newly diagnosed. No credits applied."
    }
  ]
}
```
"""


In [ ]:
# 4) Build the agent and helpers (Life-only)
import re


agent = Agent(system_prompt=LIFE_PROMPT, tools=[kb_search, calculator], model=MODEL_ID)


def run_scoring_agent(impairments: dict) -> dict:
    payload = json.dumps(impairments, ensure_ascii=False)
    res = agent(payload)
    res_str = str(res)
    # Extract fenced JSON if present
    fence = re.search(r"```json\s*(.*?)\s*```", res_str, re.DOTALL)
    if fence:
        res_str = fence.group(1)
        return json.loads(res_str)
    else:
        return {"error": "No JSON found in response"}
    


In [ ]:
# 5) Sample impairments payload and local evaluation
sample_impairments_life = [
    {
        "impairment_id": "hypertension",
        "scoring_factors": {
            "blood_pressure": "128/92 mmHg",
            "age": 41,
            "medication": "Lisinopril 10mg"
        },
        "evidence": [
            "Application: Blood pressure reading 128/92 mmHg (Page 14)",
            "Rx: Lisinopril 10mg (Page 10,11)"
        ]
    },
    {
        "impairment_id": "diabetes",
        "scoring_factors": {
            "A1C": 8.2,
            "treatment": "Insulin"
        },
        "evidence": [
            "Lab: A1C 8.2 % (Page 1)",
            "Rx: Insulin (Page 3)"
        ]
    }
]


### Input to the scoring agent: impairments from Lab 2
This array mirrors the JSON emitted by the Impairment Detection agent you built in Lab 2. Each item includes a canonical `impairment_id`, a set of `scoring_factors` extracted and normalized against the Knowledge Base, and supporting `evidence`. In production, the scoring agent consumes this payload directly from the prior step in the workflow to compute per‑impairment subtotals and a total score.


Now let's run the scoring agent on the sample impairments payload.

In [ ]:


life_result = run_scoring_agent(sample_impairments_life)
print(json.dumps(life_result, indent=2)[:2000])


### Interpreting the scoring output

The agent displays its step-by-step reasoning before returning the final score. You'll see it calling tools:
- **`kb_search`**: Retrieves the underwriting manual section for each impairment to access rating tables and scoring guidelines
- **`calculator`**: Performs deterministic math to sum debits and credits, ensuring accurate arithmetic without "creative math" from the LLM

The agent then emits a structured JSON object:

**`total_score`** — The aggregate risk score across all impairments. This is the primary number used for underwriting decisions (e.g., approve, rate up, decline).

**`impairment_scores[]`** — Per-impairment breakdown, each containing:
- **`impairment_id`**: Matches the canonical ID from the detection agent (Lab 2)
- **`sub_total`**: The risk score for this specific impairment, calculated by applying the scoring factors to the rating tables in the underwriting manual
- **`reason`**: A human-readable explanation citing the exact factors used (e.g., "A1C 8.2%, insulin treatment") and the specific rating table rows/columns consulted. This provides full transparency into how the score was derived.

**Why this structure matters:**
- **Auditable**: Underwriters can trace each score back to specific policy tables and factors, verifying the AI's math and logic
- **Machine-consumable**: The structured JSON feeds directly into the workflow (Step Functions) and UI, enabling automated routing decisions (e.g., auto-approve scores below threshold, flag high-risk cases for manual review)
- **Explainable**: The `reason` field provides the narrative justification required for regulatory compliance and customer communication

# Deploying to Amazon Bedrock AgentCore Runtime
Great! Now let's deploy the scoring agent to Amazon Bedrock AgentCore Runtime.

Amazon Bedrock AgentCore Runtime is a secure, serverless environment for hosting and scaling AI agents. It manages infrastructure concerns like scaling, session management, and security isolation, so we can focus on the agent’s logic and capabilities. Deploying here gives us a dedicated, observable endpoint that integrates cleanly with other AWS services and our workflow/UI.

In the next cells we will:
- Generate a small runtime file with an @entrypoint (the callable the runtime exposes)
- Configure the runtime using the Starter Toolkit (entrypoint, execution role, region, ECR)
- Build and deploy the agent container with CodeBuild (no local Docker required)
- Wait for the runtime endpoint to become READY
- Invoke the endpoint with a sample impairments payload to verify end-to-end scoring


### Execution role for the runtime
The runtime needs an execution role to pull container images, write logs, and invoke Bedrock resources securely. Creating it here keeps permissions explicit and least‑privileged, parameterized for your account and region, and avoids hidden console dependencies so deployment is repeatable from the notebook.


In [ ]:
agent_code = f"""
from strands import Agent, tool
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import os, json
import boto3
from botocore.config import Config
import re


# Minimal env/clients
REGION = "{REGION}"
BEDROCK_KB_ID = "{BEDROCK_KB_ID}"
FOUNDATION_MODEL_ID = "{FOUNDATION_MODEL_ID}" 
session = boto3.session.Session(region_name=REGION)
kb_runtime = session.client('bedrock-agent-runtime')

@tool
def kb_search(canonical_term: str):
    print(f"[kb_search] Searching for {{canonical_term}}")
    try:
        resp = kb_runtime.retrieve(
            knowledgeBaseId=BEDROCK_KB_ID,
            retrievalQuery={{"text": canonical_term}},
            retrievalConfiguration={{"vectorSearchConfiguration": {{"numberOfResults": 1}}}}
        )
        results = resp.get('retrievalResults') or []
        if not results:
            return "No matching documents found."
        content = results[0].get('content').get('text') or {{}}
        location = results[0].get('location').get('s3Location').get('uri') or {{}}
        print(f"[kb_search] Found {{canonical_term}} in {{location}}")
        # Bedrock returns {{'text': '...'}} per docs
        return f\"\"\"
        knowledgebase_location: {{location}}
        text_content: {{content}}
        \"\"\"
    except Exception as e:
        return f"KB retrieval error: {{e}}"

SYSTEM_PROMPT = \"\"\"
{LIFE_PROMPT}
\"\"\"

retrying_cfg = Config(
    retries={"mode": "adaptive", "max_attempts": 12}
)
model = BedrockModel(
    model_id=FOUNDATION_MODEL_ID,
    boto_client_config=retrying_cfg
)
agent = Agent(model=model, tools=[kb_search], system_prompt=SYSTEM_PROMPT)

app = BedrockAgentCoreApp()

@app.entrypoint
def strands_impairment_agent(payload: dict):
    print("payload")
    print(payload)
    raw = payload.get('impairments')
    message = json.dumps(raw, ensure_ascii=False)
    res = agent(message)
    res_str = str(res)
    fence = re.search(r"```json\\s*(.*?)\\s*```", res_str, re.DOTALL)
    if fence:
        json_res = fence.group(1)
        return json.loads(json_res)
    else:
        return {{"error": "No JSON found in response"}}
    
    return json_res


if __name__ == "__main__":
    app.run()
"""

from pathlib import Path
Path("strands_scoring_runtime.py").write_text(agent_code, encoding="utf-8")


In [ ]:
%%writefile requirements.txt
strands-agents
bedrock-agentcore
bedrock-agentcore-starter-toolkit
boto3
python-dotenv


In [ ]:
# Create IAM Role for AgentCore Runtime (Life-only)
import json
import time
import boto3
from botocore.exceptions import ClientError

# Discover account and region
sts = boto3.client('sts')
account_id = sts.get_caller_identity().get('Account')
region_for_arns = REGION  # Reuse REGION from earlier env cell

role_name = "uw_scoring_agent_exec"
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": ["bedrock-agentcore.amazonaws.com"]},
            "Action": "sts:AssumeRole"
        }
    ]
}

policy_doc = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "ECRImageAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:BatchGetImage",
                "ecr:GetDownloadUrlForLayer"
            ],
            "Resource": [
                f"arn:aws:ecr:{region_for_arns}:{account_id}:repository/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogStreams",
                "logs:CreateLogGroup"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:DescribeLogGroups"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": [
                f"arn:aws:logs:{region_for_arns}:{account_id}:log-group:/aws/bedrock-agentcore/runtimes/*:log-stream:*"
            ]
        },
        {
            "Sid": "ECRTokenAccess",
            "Effect": "Allow",
            "Action": [
                "ecr:GetAuthorizationToken"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "xray:PutTraceSegments",
                "xray:PutTelemetryRecords",
                "xray:GetSamplingRules",
                "xray:GetSamplingTargets"
            ],
            "Resource": ["*"]
        },
        {
            "Effect": "Allow",
            "Resource": "*",
            "Action": "cloudwatch:PutMetricData",
            "Condition": {
                "StringEquals": {
                    "cloudwatch:namespace": "bedrock-agentcore"
                }
            }
        },
        {
            "Sid": "BedrockAgentCoreRuntime",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:InvokeAgentRuntime"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:runtime/*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreMemoryCreateMemory",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateMemory"
            ],
            "Resource": "*"
        },
        {
            "Sid": "BedrockAgentCoreMemory",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:CreateEvent",
                "bedrock-agentcore:GetEvent",
                "bedrock-agentcore:GetMemory",
                "bedrock-agentcore:GetMemoryRecord",
                "bedrock-agentcore:ListActors",
                "bedrock-agentcore:ListEvents",
                "bedrock-agentcore:ListMemoryRecords",
                "bedrock-agentcore:ListSessions",
                "bedrock-agentcore:DeleteEvent",
                "bedrock-agentcore:DeleteMemoryRecord",
                "bedrock-agentcore:RetrieveMemoryRecords"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:memory/*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetResourceApiKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default/apikeycredentialprovider/*",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetResourceOauth2Token",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceOauth2Token"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:token-vault/default/oauth2credentialprovider/*",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockAgentCoreIdentityGetWorkloadAccessToken",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetWorkloadAccessToken",
                "bedrock-agentcore:GetWorkloadAccessTokenForJWT",
                "bedrock-agentcore:GetWorkloadAccessTokenForUserId"
            ],
            "Resource": [
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default",
                f"arn:aws:bedrock-agentcore:{region_for_arns}:{account_id}:workload-identity-directory/default/workload-identity/uw_impairment_agent-*"
            ]
        },
        {
            "Sid": "BedrockModelInvocation",
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel",
                "bedrock:InvokeModelWithResponseStream",
                "bedrock:ApplyGuardrail"
            ],
            "Resource": [
                "arn:aws:bedrock:*::foundation-model/*",
                f"arn:aws:bedrock:{region_for_arns}:{account_id}:*"
            ]
        },
        {
            "Sid": "RAG",
            "Effect": "Allow",
            "Action": [
                "bedrock:RetrieveAndGenerate",
                "bedrock:Retrieve"
            ],
            "Resource": "*"
        }
    ]
}

iam = boto3.client('iam')
role_arn = None
try:
    resp = iam.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy),
        Description="Execution role for Bedrock AgentCore runtime (Scoring Agent)",
        MaxSessionDuration=3600
    )
    role_arn = resp['Role']['Arn']
    # Small delay for policy propagation
    time.sleep(2)
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName=role_name)['Role']['Arn']
    else:
        raise

# Put/Update inline policy
iam.put_role_policy(
    RoleName=role_name,
    PolicyName="uw_scoring_agent_inline",
    PolicyDocument=json.dumps(policy_doc)
)

print(f"✅ IAM Role ready: {role_arn}")


In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "uw_scoring_agent"
response = agentcore_runtime.configure(
    entrypoint="strands_scoring_runtime.py",
    execution_role=role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response


In [ ]:
# Launch the AgentCore Runtime (cloud build)
launch_result = agentcore_runtime.launch()
launch_result


In [ ]:
# Wait for runtime to be READY
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status


In [ ]:
# Invoke the runtime using the Starter Toolkit helper (Life-only)
invoke_payload = {
    "impairments": sample_impairments_life
}
invoke_response = agentcore_runtime.invoke(invoke_payload)
invoke_response.get("response")[0]


In [ ]:
#print arn
print(f"Scoring Agent ARN: {launch_result.agent_arn}")

### Workshop integration: update the workflow Lambda environment

If you're completing this lab as part of the full workshop, update the Step Functions–driven Lambda to point at your deployed agent.

- Open the AWS Lambda console (us‑east‑1): [AWS Lambda Console](https://console.aws.amazon.com/lambda/home?region=us-east-1#/functions)
- For this lab, update the Lambda function: `score-lambda`
- In the function's Configuration → Environment variables, set `AGENT_ARN` to your deployed Scoring Agent ARN from this notebook, then Save.

Note: Function names are defined in `static/underwriting-workshop.yaml`.
